In [ ]:
# loading and normalizing cifar10
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np

import torch.nn as nn
import torch.nn.functional as F

from torchsummary import summary

In [ ]:

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])


In [ ]:
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
                                        
                                        
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=32,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [ ]:
## creating a function that initializes weights
def init_weight_conv(m):
    if type(m) == nn.Conv2d:
        torch.nn.init.kaiming_normal_(m.weight) #he initialize, can use xavier instead
        m.bias.data.fill_(0.001) # optional bias
        
def init_weight_linear(m):
    if type(m) == nn.Linear:
        torch.nn.init.kaiming_normal_(m.weight)
        m.bias.data.fill_(0.001)
    

In [ ]:
# inception style module
class convnet(nn.Module):
    def __init__(self):
        super(convnet, self).__init__()
        self.conv_3x3 = nn.Sequential(
            nn.Conv2d(3, 8, 1, padding = 0),
            nn.BatchNorm2d(8, eps=0.001), # outchannels match previous conv layer
            nn.ReLU(),
            nn.Conv2d(8, 8, 3, padding = 1),
            nn.BatchNorm2d(8, eps=0.001),
            nn.ReLU()
        )
        self.conv_5x5 = nn.Sequential(
            nn.Conv2d(3, 8, 1, padding = 0),
            nn.BatchNorm2d(8, eps=0.001),
            nn.ReLU(),
            nn.Conv2d(8, 8, 5, padding = 2),
            nn.BatchNorm2d(8, eps=0.001),
            nn.ReLU()
        )
        self.max_pool_conv_1x1 = nn.Sequential(
            nn.MaxPool2d(3, stride = 1, padding=1), # window size 3x3, stride and padding = 1 to ensure dimensionality matches
            nn.Conv2d(3, 8, 1, padding = 0),
            nn.BatchNorm2d(8, eps=0.001),
            nn.ReLU()
        )
        self.conv_1x1 = nn.Sequential(
            nn.Conv2d(3, 8, 1, padding = 0),
            nn.BatchNorm2d(8, eps=0.001),
            nn.ReLU()
        )
        self.fc1 = nn.Linear(8192, 10) 
        #self.dropout = nn.Dropout(0.5)
        #self.fc2 = nn.Linear(32, 10)
        
    def forward(self, x):
        x1 = self.conv_1x1(x)
        x2 = self.conv_3x3(x)
        x3 = self.conv_5x5(x)
        x4 = self.max_pool_conv_1x1(x)
        x = torch.cat([x1,x2,x3,x4], 1) # concatenate all layers
        x = F.avg_pool2d(x, stride = 2, kernel_size= 2)
        x = x.view(x.size(0), -1) # flatten
        x = self.fc1(x)
        #x = self.dropout(x)
        #x = self.fc2(x)
        return x
        
net = convnet()
            
            

            

In [ ]:
# visualize network
#for i, weights in enumerate(list(net.parameters())):
#    print('i:',i,'weights:',weights.size())

#print(net)


summary(net, (3, 32, 32))


In [ ]:
# apply initializers
net.apply(init_weight_conv)
net.apply(init_weight_linear)

In [ ]:
# define loss and optimizer
import torch.optim as optim
learning_rate = 0.001
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=learning_rate) #net.parameters

In [ ]:
# modified model training to keep track of train/val loss
n_epochs = 2

for epoch in range(n_epochs):
    running_loss = 0.0
    for i, train_data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = train_data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print loss per n minibatches
        running_loss += loss.item()
        if i % 50 == 49:    # print every 50 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 50))
            running_loss = 0.0
    
    # keep track of loss in test dataset 
    correct = 0
    total = 0
    running_test_loss = 0.0
    with torch.no_grad():
        for test_data in testloader:
            test_images, test_labels = test_data
            test_outputs = net(test_images)
            test_loss = criterion(test_outputs, test_labels)
            running_test_loss += test_loss.item()
            _, predicted = torch.max(test_outputs.data, 1)
            total += test_labels.size(0)
            correct += (predicted == test_labels).sum().item()


    
    
    # for printing average loss every epoch
    print("===> Epoch {} Complete: Train Avg. Loss: {:.4f}".format(epoch+1, running_loss / len(trainloader)))
    print("===> Epoch {} Complete: Test Avg. Loss: {:.4f}".format(epoch+1, running_test_loss / len(testloader)))
    print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))
print('Finished Training')